In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

class NeutrinoOverdensityCalculator:
    def __init__(self, c_code_file):
        self.c_code_file = c_code_file
        self.executable = "nh"

    def compile_c_code(self):
    # Compile the C code using GCC 
        subprocess.run(["gcc", self.c_code_file, "-o", self.executable, "-lm", "-O3"], check=True)

    def run_c_code(self, M12, zo, c, mnu):
        # Execute the compiled C code with the provided input parameters
        result = subprocess.run([f"./{self.executable}", str(M12), str(zo), str(c), str(mnu)],
                                stdout=subprocess.PIPE, text=True, check=True)
        return result.stdout

    def calculate(self, M12, zo, c, mnu):
        # Compile the C code
        self.compile_c_code()
    
        # Run the C code and save the output
        output = self.run_c_code(M12, zo, c, mnu)
    
        # Organize the output 
        lines = output.strip().split("\n")
        radii = []
        densities = []
        for line in lines:
            try:
                # Attempt to split and convert to floats
                radius, density = map(float, line.split())
                radii.append(radius)
                densities.append(density)
            except ValueError:
            # Skip lines that cant be converted to floats (numerical errors, etc.)
                continue
    
        radii = np.array(radii)
        densities = np.array(densities)
        return radii, densities


if __name__ == "__main__":
    c_code_file = "nh.c"
    calculator = NeutrinoOverdensityCalculator(c_code_file)

    # Input parameters
    M12 = 1e3   # Halo mass in units of 10^12 Msun
    zo = 0.0    # Observed redshift
    c = 4.5    # Concentration parameter
    mnu = 0.3  # Neutrino mass in eV


    start_time = time.time()
    # Compiling and running the code
    radii, densities = calculator.calculate(M12, zo, c, mnu)
    end_time = time.time()

    elapsed_time = end_time - start_time
    print(f"Time elapsed:{elapsed_time:.2f} seconds")

    # Plot the results
    plt.loglog(radii, densities)
    plt.xlabel('r (Mpc / h)', fontsize = 12)
    plt.ylabel(r'$n_{\nu}$ / $\overline{n}_{\nu}$', fontsize = 13)
    plt.hlines(1, radii[0]/2, radii[-2]*2, linestyle = 'dotted', alpha=0.5)
    #print("Radii:", radii)
    #print("Densities:", densities)

#### Integrands Contour ####

In [ ]:
num_velocities = 100
num_angles = 5
v_values = np.linspace(0, 12, num_velocities)  # Adjust for low neutrino masses
mu_values = np.linspace(-1, 1, num_angles)  # Example: mu range from -1 to 1

# Load in integrand file
filename = 'integrand.out'
df = pd.read_csv(filename, delim_whitespace=True, header=None,
                 names=['r', 'mu', 'sqrt_Phi', 'x', 'v', 'integrand'])

# Initialize 3D array for the integrand values
integrand_reshaped = np.zeros((100, num_angles, num_velocities))  # 100 radial positions

# Total number of data points per radial position
points_per_radial = num_velocities * num_angles

# Process integrands
for radial_index in range(100):  # Assuming 100 radial positions, adjust accordingly 
    start_idx = radial_index * points_per_radial
    end_idx = start_idx + points_per_radial

    # Extract and reshape data for the radial position
    df_radial = df.iloc[start_idx:end_idx]
    integrand_values = df_radial['integrand'].values.reshape(num_angles, num_velocities)
    integrand_reshaped[radial_index] = integrand_values  

# Function to plot contour for a specific radial position
def plot_contour_for_radial_position(radial_index):
    if radial_index < 0 or radial_index >= 100:
        raise ValueError("Radial index out of range. It should be between 0 and 99.")
    
    integrand_values = integrand_reshaped[radial_index]
    plt.figure(figsize=(8, 6))
    X, Y = np.meshgrid(v_values, mu_values)
    contour = plt.contourf(X, Y, integrand_values, levels=50, cmap='viridis')

    plt.colorbar(contour, label='Integrand Value')
    plt.xlabel(r'Velocity (x)')
    plt.ylabel(r'$\mu = \cos(\phi)$')
    plt.title(f'Integrands Contour Plot for Radial Position {radial_index}')
    plt.xlim(v_values.min(), v_values.max())
    plt.ylim(mu_values.min(), mu_values.max())
    
    plt.show()

# Specify radial index 
plot_contour_for_radial_position(98)